1. Every 15 minutes call the api again to get last 15 minutes of data
2. Make Image with data
3. Run Image through model
4. Send results to trading loop
5. Append data to dataframe

In [9]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import requests
import time
import datetime
import json
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import cv2 
from PIL import Image
import mplfinance as mpf
import io
import torch
import torchvision.transforms as transforms


In [10]:
area = (120, 69, 665, 472)

model = torch.load('res_mod.pt')
model.eval().cpu()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [11]:
def transform_image(image_bytes):
    my_transforms = transforms.Compose([
        transforms.Resize((256,256)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    image = Image.open(io.BytesIO(image_bytes))
    return my_transforms(image).unsqueeze(0)

In [12]:
def minute_scraper(now, then):
    url = 'https://api.pro.coinbase.com/products/ETH-USD/candles'
    parameters = {
        'start':then,
        'end':now,
        'granularity': '60'
     }
    r = requests.get(url, parameters)
    firsteth = pd.DataFrame(r.json())
    firsteth = firsteth.rename(columns={1: "Low", 2: "High", 3: "Open", 4: "Close", 5: "Volume"})
    firsteth['datetime'] = pd.to_datetime(firsteth[0], unit='s')\
           .dt.strftime('%Y-%m-%d %H:%M:%S')
    firsteth = firsteth.set_index( pd.DatetimeIndex(firsteth['datetime']))
    firsteth = firsteth.drop(['datetime', 0], axis=1)
    firsteth = firsteth[::-1]
    return firsteth



In [13]:
    
now = str(datetime.datetime.now())
then = str(datetime.datetime.now() - datetime.timedelta(minutes=15))
eth15 = minute_scraper(now, then)

name = f'i{now}.png'
kwargs = dict(type='candle',style='yahoo', mav=(2,4,6),volume=True,figratio=(11,8),figscale=1)
mpf.plot(eth15,**kwargs,savefig=f'images/{name}')
img = Image.open(f"images/{name}")
cropped_img = img.crop(area)
cropped_img.save(f"images/{name}") 

im = cv2.imread(f"images/{name}")
im_resize = cv2.resize(im, (256, 256))

is_success, im_buf_arr = cv2.imencode(".png", im_resize)
byte_im = im_buf_arr.tobytes()
tensor = transform_image(byte_im)
print(int(torch.argmax(model(tensor))))


0
